In [8]:
#SQL CheatSheet
#https://www.stratascratch.com/blog/sql-cheat-sheet-technical-concepts-for-the-job-interview/

# 1. Most Profitable Companies


Forbes
Medium
ID 10354


Find the 3 most profitable companies in the entire world.
Output the result along with the corresponding company name.
Sort the result based on profits in descending order.


# 2.Workers With The Highest Salaries


Interview Question Date: July 2021

Amazon
DoorDash
Easy
ID 10353


You have been asked to find the job titles of the highest-paid employees.


Your output should include the highest-paid title or multiple titles with the same salary.

DataFrames: worker, titleExpected Output Type: pandas.DataFrame

# 3. Users By Average Session Time


Interview Question Date: July 2021

Meta/Facebook
Medium
ID 10352

Calculate each user's average session time. A session is defined as the time difference between a page_load and page_exit. For simplicity, assume a user has only 1 session per day and if there are multiple of the same events on that day, consider only the latest page_load and earliest page_exit, with an obvious restriction that load time event should happen before exit time event . Output the user_id and their average session time.

Table: facebook_web_log

# 4.Activity Rank


Interview Question Date: July 2021

Google
Medium
ID 10351

Find the email activity rank for each user. Email activity rank is defined by the total number of emails sent. The user with the highest number of emails sent will have a rank of 1, and so on. Output the user, total emails, and their activity rank. Order records by the total emails in descending order. Sort users with the same number of emails in alphabetical order.
In your rankings, return a unique value (i.e., a unique rank) even if multiple users have the same number of emails. For tie breaker use alphabetical order of the user usernames.

Table: google_gmail_emails

# 5.Algorithm Performance


Interview Question Date: July 2021

Meta/Facebook
Hard


Meta/Facebook is developing a search algorithm that will allow users to search through their post history. You have been assigned to evaluate the performance of this algorithm.


We have a table with the user's search term, search result positions, and whether or not the user clicked on the search result.


Write a query that assigns ratings to the searches in the following way:
•	If the search was not clicked for any term, assign the search with rating=1
•	If the search was clicked but the top position of clicked terms was outside the top 3 positions, assign the search a rating=2
•	If the search was clicked and the top position of a clicked term was in the top 3 positions, assign the search a rating=3


As a search ID can contain more than one search term, select the highest rating for that search ID. Output the search ID and its highest rating.


Example: The search_id 1 was clicked (clicked = 1) and its position is outside of the top 3 positions (search_results_position = 5), therefore its rating is 2.

Table: fb_search_events

In [ ]:
--法2：
SELECT 
    search_id,
    MAX(CASE WHEN clicked = 0 THEN 1
        WHEN search_results_position > 3 THEN 2
        ELSE 3 END) as max_rating
FROM fb_search_events
GROUP BY search_id

# 6.Distances Traveled


Interview Question Date: December 2020

Lyft
Medium
ID 10324

Find the top 10 users that have traveled the greatest distance. Output their id, name and a total distance traveled.

Tables: lyft_rides_log, lyft_users

# 7.Finding User Purchases【有个video可看】


Interview Question Date: December 2020

Amazon
Medium
ID 10322

Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user_ids of these returning active users.

Table: amazon_transactions

# 8.Monthly Percentage Difference


Interview Question Date: December 2020

Amazon
Hard
ID 10319

Given a table of purchases by date, calculate the month-over-month percentage change in revenue. The output should include the year-month date (YYYY-MM) and percentage change, rounded to the 2nd decimal point, and sorted from the beginning of the year to the end of the year.
The percentage change column will be populated from the 2nd month forward and can be calculated as ((this month's revenue - last month's revenue) / last month's revenue)*100.

Table: sf_transactions

# 9.New Products


Interview Question Date: December 2020

Salesforce
Tesla
Medium
ID 10318

You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year.

# 10. Cities With The Most Expensive Homes


Interview Question Date: December 2020

Zillow
Medium
ID 10315

Write a query that identifies cities with higher than average home prices when compared to the national average. Output the city names.

Table: zillow_transactions

# 11.Revenue Over Time


Interview Question Date: December 2020

Amazon
Hard
ID 10314

Find the 3-month rolling average of total revenue from purchases given a table with users, their purchase amount, and date purchased. Do not include returns which are represented by negative purchase values. Output the year-month (YYYY-MM) and 3-month rolling average of revenue, sorted from earliest month to latest month.


A 3-month rolling average is defined by calculating the average total revenue from all user purchases for the current month and previous two months. The first two months will not be a true 3-month rolling average since we are not given data from last year. Assume each month has at least one purchase.

In [ ]:
#本月及前两个月的rolling avg：
#3 months rolling avg：avg total rev from all purchases for current and preivous 2 months：
AVG(t.monthly_revenue) OVER(ORDER BY t.month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS avg_revenue

# 12.Naive Forecasting


Interview Question Date: December 2020

Uber
Hard
ID 10313

Some forecasting methods are extremely simple and surprisingly effective. Naïve forecast is one of them; we simply set all forecasts to be the value of the last observation. Our goal is to develop a naïve forecast for a new metric called "distance per dollar" defined as the (distance_to_travel/monetary_cost) in our dataset and measure its accuracy.


To develop this forecast,  sum "distance to travel"  and "monetary cost" values at a monthly level before calculating "distance per dollar". This value becomes your actual value for the current month. The next step is to populate the forecasted value for each month. This can be achieved simply by getting the previous month's value in a separate column. Now, we have actual and forecasted values. This is your naïve forecast. Let’s evaluate our model by calculating an error matrix called root mean squared error (RMSE). RMSE is defined as sqrt(mean(square(actual - forecast)). Report out the RMSE rounded to the 2nd decimal spot.

Table: uber_request_logs

# 13.Class Performance


Interview Question Date: December 2020

Box
Medium
ID 10310

You are given a table containing assignment scores of students in a class. Write a query that identifies the largest difference in total score  of all assignments.
Output just the difference in total score (sum of all 3 assignments) between a student with the highest score and a student with the lowest score.

Table: box_scores

# 14. Salaries Differences


Interview Question Date: November 2020

LinkedIn
Dropbox
Easy
ID 10308

Write a query that calculates the difference between the highest salaries found in the marketing and engineering departments. Output just the absolute difference in salaries.

Tables: db_employee, db_dept

# 15.Risky Projects


Interview Question Date: November 2020

LinkedIn
Medium
ID 10304

Identify projects that are at risk for going overbudget. A project is considered to be overbudget if the cost of all employees assigned to the project is greater than the budget of the project.


You'll need to prorate the cost of the employees to the duration of the project. For example, if the budget for a project that takes half a year to complete is 10K, then the total half-year salary of all employees assigned to the project should not exceed $10K. Salary is defined on a yearly basis, so be careful how to calculate salaries for the projects that last less or more than one year.


Output a list of projects that are overbudget with their project name, project budget, and prorated total employee expense (rounded to the next dollar amount).


HINT: to make it simpler, consider that all years have 365 days. You don't need to think about the leap years.

Tables: linkedin_projects, linkedin_emp_projects, linkedin_employees

# 16.Top Percentile Fraud


Interview Question Date: November 2020

Google
Netflix
Hard
ID 10303

ABC Corp is a mid-sized insurer in the US and in the recent past their fraudulent claims have increased significantly for their personal auto insurance portfolio. They have developed a ML based predictive model to identify propensity of fraudulent claims. Now, they assign highly experienced claim adjusters for top 5 percentile of claims identified by the model.
Your objective is to identify the top 5 percentile of claims from each state. Your output should be policy number, state, claim cost, and fraud score.

Table: fraud_score

# 17. Distance Per Dollar


Interview Question Date: November 2020

Uber
Hard
ID 10302

You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.


The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal).
You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.

Table: uber_request_logs

# 18. Expensive Projects


Interview Question Date: November 2020

Microsoft
Medium
ID 10301

Given a list of projects and employees mapped to each project, calculate by the amount of project budget allocated to each employee . The output should include the project title and the project budget rounded to the closest integer. Order your list by projects with the highest budget per employee first.

Tables: ms_projects, ms_emp_projects

# 19.Premium vs Freemium


Interview Question Date: November 2020

Microsoft
Hard
ID 10300

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

Tables: ms_user_dimension, ms_acc_dimension, ms_download_facts

# 20. Finding Updated Records


Interview Question Date: November 2020

Microsoft
Easy

We have a table with employees and their salaries, however, some of the records are old and contain outdated salary information. Find the current salary of each employee assuming that salaries increase each year. Output their id, first name, last name, department ID, and current salary. Order your list by employee ID in ascending order.

Table: ms_employee_salary

# 21.Comments Distribution


Interview Question Date: November 2020

Meta/Facebook
Hard
ID 10297

Write a query to calculate the distribution of comments by the count of users that joined Meta/Facebook between 2018 and 2020, for the month of January 2020.


The output should contain a count of comments and the corresponding number of users that made that number of comments in Jan-2020. For example, you'll be counting how many users made 1 comment, 2 comments, 3 comments, 4 comments, etc in Jan-2020. Your left column in the output will be the number of comments while your right column in the output will be the number of users. Sort the output from the least number of comments to highest.


To add some complexity, there might be a bug where an user post is dated before the user join date. You'll want to remove these posts from the result.

Tables: fb_users, fb_comments

# 22.Most Active Users On Messenger


Interview Question Date: November 2020

Meta/Facebook
Medium
ID 10295

Meta/Facebook Messenger stores the number of messages between users in a table named 'fb_messages'. In this table 'user1' is the sender, 'user2' is the receiver, and 'msg_count' is the number of messages exchanged between them.
Find the top 10 most active users on Meta/Facebook Messenger by counting their total number of messages sent and received. Your solution should output usernames and the count of the total messages they sent or received

Table: fb_messages

# 23.SMS Confirmations From Users


Interview Question Date: November 2020

Meta/Facebook
Medium

Meta/Facebook sends SMS texts when users attempt to 2FA (2-factor authenticate) into the platform to log in. In order to successfully 2FA they must confirm they received the SMS text message. Confirmation texts are only valid on the date they were sent.


Unfortunately, there was an ETL problem with the database where friend requests and invalid confirmation records were inserted into the logs, which are stored in the 'fb_sms_sends' table. These message types should not be in the table.


Fortunately, the 'fb_confirmers' table contains valid confirmation records so you can use this table to identify SMS text messages that were confirmed by the user.


Calculate the percentage of confirmed SMS texts for August 4, 2020. Be aware that there are multiple message types, the ones you're interested in are messages with type equal to 'message'.

Tables: fb_sms_sends, fb_confirmers

# 24. Acceptance Rate By Date


Interview Question Date: November 2020

Meta/Facebook
Medium
ID 10285

What is the overall friend acceptance rate by date? Your output should have the rate of acceptances by the date the request was sent. Order by the earliest date to latest.


Assume that each friend request starts by a user sending (i.e., user_id_sender) a friend request to another user (i.e., user_id_receiver) that's logged in the table with action = 'sent'. If the request is accepted, the table logs action = 'accepted'. If the request is not accepted, no record of action = 'accepted' is logged.

# 25. Popularity Percentage


Interview Question Date: November 2020

Meta/Facebook
Hard
ID 10284

Find the popularity percentage for each user on Meta/Facebook. The popularity percentage is defined as the total number of friends the user has divided by the total number of users on the platform, then converted into a percentage by multiplying by 100.
Output each user along with their popularity percentage. Order records in ascending order by user id.
The 'user1' and 'user2' column are pairs of friends.

Table: facebook_friends

# 26.Find the top-ranked songs for the past 20 years.


Spotify
Medium
ID 10283

Find all the songs that were top-ranked (at first position) at least once in the past 20 years

Table: billboard_top_100_year_end

# 27.Find all inspections which are part of an inactive program
City of Los Angeles
Easy
ID 10277

Find all inspections which are part of an inactive program.

# 28. Find the total number of available beds per hosts' nationality


Airbnb
Medium
ID 10187

Find the total number of available beds per hosts' nationality.
Output the nationality along with the corresponding total number of available beds.
Sort records by the total available beds in descending order.

Tables: airbnb_apartments, airbnb_hosts

# 29.Order all countries by the year they first participated in the Olympics


ESPN
Easy
ID 10184

Order all countries by the year they first participated in the Olympics.

Output the National Olympics Committee (NOC) name along with the desired year.

Sort records by the year and the NOC in ascending order.

Table: olympics_athletes_events

# 30.Total Cost Of Orders


Interview Question Date: July 2020

Amazon
Etsy
Easy
ID 10183

Find the total cost of each customer's orders. Output customer's id, first name, and the total order cost. Order records by customer's first name alphabetically.

Tables: customers, orders

# 31.Find the lowest score for each facility in Hollywood Boulevard


Interview Question Date: July 2020

City of Los Angeles
City of San Francisco
Tripadvisor
Medium
ID 10180

Find the lowest score per each facility in Hollywood Boulevard.
Output the result along with the corresponding facility name.
Order the result based on the lowest score in descending order and the facility name in the ascending order.

Table: los_angeles_restaurant_health_inspections

# 32.Businesses Open On Sunday


Yelp
Medium
ID 10178

Find the number of businesses that are open on Sundays. Output the slot of operating hours along with the corresponding number of businesses open during those time slots. Order records by total number of businesses opened during those hours in descending order.

Tables: yelp_business_hours, yelp_business

# 33.Bikes Last Used


Lyft
DoorDash
Easy
ID 10176

Find the last time each bike was in use. Output both the bike number and the date-timestamp of the bike's last use (i.e., the date-time the bike was returned). Order the results by bikes that were most recently used.

Table: dc_bikeshare_q1_2012

# 34.Days At Number One


Spotify
Hard
ID 10173

Find the number of days a US track has stayed in the 1st position for both the US and worldwide rankings. Output the track name and the number of days in the 1st position. Order your output alphabetically by track name.


If the region 'US' appears in dataset, it should be included in the worldwide ranking.

Tables: spotify_daily_rankings_2017_us, spotify_worldwide_daily_song_ranking

# 35.Best Selling Item


Interview Question Date: July 2020

Amazon
Ebay
Best Buy
Hard
ID 10172

Find the best selling item for each month (no need to separate months by year) where the biggest total invoice was paid. The best selling item is calculated using the formula (unitprice * quantity). Output the description of the item along with the amount paid.

Table: online_retail

# 36.Find the genre of the person with the most number of oscar winnings


Netflix
Hard
ID 10171

Find the genre of the person with the most number of oscar winnings.
If there are more than one person with the same number of oscar wins, return the first one in alphabetic order based on their name. Use the names as keys when joining the tables.

Tables: oscar_nominees, nominee_information

In [ ]:
求user的count, count最大值记录对应的属性：
i.求每个演员得奖几次，对应的排名：name, count(*), rank() over (order by COUNT(*) DESC)；
ii.求genre让名字从刚才的query中取rank1：name IN (select nominess from wins where rnk = 1)

nominee, COUNT(*) OVER (PARTITION BY nominee)  = nominee, COUNT(*), R + GROUP BY 1

# 37.Gender With Most Doctor Appointments


HealthTap
Natera
Easy
ID 10170

Find the gender that has made the most number of doctor appointments.
Output the gender along with the corresponding number of appointments.

Table: medical_appointments

# 38.Highest Total Miles


Interview Question Date: July 2020

Uber
Medium
ID 10169

You’re given a table of Uber rides that contains the mileage and the purpose for the business expense.  You’re asked to find business purposes that generate the most miles driven for passengers that use Uber for their business transportation. Find the top 3 business purpose categories by total mileage.

Table: my_uber_drives

# 39.Number Of Records By Variety


Microsoft
Linux
Easy
ID 10168

Find the total number of records that belong to each variety in the dataset. Output the variety along with the corresponding number of records. Order records by the variety in ascending order.

Table: iris

# 40.Total Number Of Housing Units


Airbnb
Zillow
Easy
ID 10167

Find the total number of housing units completed for each year. Output the year along with the total number of housings. Order the result by year in ascending order.


Note: Number of housing units in thousands.

Table: housing_units_completed_us

# 41.Reviews of Hotel Arena


Airbnb
Expedia
Easy
ID 10166

Find the number of rows for each review score earned by 'Hotel Arena'. Output the hotel name (which should be 'Hotel Arena'), review score along with the corresponding number of rows with that score for the specified hotel.

Table: hotel_reviews

# 42.Total AdWords Earnings


Interview Question Date: July 2020

Google
Easy
ID 10164

Find the total AdWords earnings for each business type. Output the business types along with the total earnings.

Table: google_adwords_earnings

# 43.Product Transaction Count


Microsoft
Nvidia
Medium
ID 10163

Find the number of transactions that occurred for each product. Output the product name along with the corresponding number of transactions and order records by the product id in ascending order. You can ignore products without transactions.

Tables: excel_sql_inventory_data, excel_sql_transaction_data

# 44.Number Of Acquisitions


Crunchbase
Easy
ID 10162

Find the number of acquisitions that occurred in each quarter of each year. Output the acquired quarter in YYYY-Qq format along with the number of acquisitions and order results by the quarters with the highest number of acquisitions first.

Table: crunchbase_acquisitions

# 45.Ranking Hosts By Beds


Interview Question Date: July 2020

Airbnb
Medium
ID 10161

Rank each host based on the number of beds they have listed. The host with the most beds should be ranked 1 and the host with the least number of beds should be ranked last. Hosts that have the same number of beds should have the same rank but there should be no gaps between ranking values. A host can also own multiple properties.
Output the host ID, number of beds, and rank from highest rank to lowest.

# 46.Rank guests based on their ages


Airbnb
Easy
ID 10160

Rank guests based on their ages.
Output the guest id along with the corresponding rank.
Order records by the age in descending order.

Table: airbnb_guests

# 47.Ranking Most Active Guests


Airbnb
Medium
ID 10159

Rank guests based on the number of messages they've exchanged with the hosts. Guests with the same number of messages as other guests should have the same rank. Do not skip rankings if the preceding rankings are identical.
Output the rank, guest id, and number of total messages they've sent. Order by the highest number of total messages first.

Table: airbnb_contacts

# 48.Number Of Units Per Nationality


Airbnb
Medium
ID 10156

Find the number of apartments per nationality that are owned by people under 30 years old.


Output the nationality along with the number of apartments.


Sort records by the apartments count in descending order.

Tables: airbnb_hosts, airbnb_units

# 49.Find the number of Yelp businesses that sell pizza


Yelp
Easy
ID 10153

Find the number of Yelp businesses that sell pizza.

Table: yelp_business

# 50. Workers With The Highest And Lowest Salaries


Amazon
Siemens
Medium
ID 10152

You have been asked to find the employees with the highest and lowest salary.


Your output should include the employee's ID, salary, and department, as well as a column salary_type that categorizes the output by:



'Highest Salary' represents the highest salary

'Lowest Salary' represents the lowest salary

Tables: worker, title

# 51.Gender With Generous Reviews


Interview Question Date: June 2020

Airbnb
Easy
ID 10149

Write a query to find which gender gives a higher average review score when writing reviews as guests. Use the from_type column to identify guest reviews. Output the gender and their average review score.

Tables: airbnb_reviews, airbnb_guests

# 52.Find the top 5 cities with the most 5 star businesses


Yelp
Medium
ID 10148

Find the top 5 cities with the most 5-star businesses. Output the city name along with the number of 5-star businesses.
In the case of multiple cities having the same number of 5-star businesses, use the ranking function returning the lowest rank in the group and output cities with a rank smaller than or equal to 5.

Table: yelp_business

# 53.Find countries that are in winemag_p1 dataset but not in winemag_p2


Interview Question Date: June 2020

Wine Magazine
Medium
ID 10147

Find countries that are in winemag_p1 dataset but not in winemag_p2.
Output distinct country names.
Order records by the country in ascending order.

Tables: winemag_p1, winemag_p2

# 54.Make a pivot table to find the highest payment in each year for each employee


City of San Francisco
Hard
ID 10145

Make a pivot table to find the highest payment in each year for each employee.
Find payment details for 2011, 2012, 2013, and 2014.
Output payment details along with the corresponding employee name.
Order records by the employee name in ascending order

Table: sf_public_salaries

# 55.Average Weight of Medal-Winning Judo


ESPN
Medium
ID 10144

Find the average weight of medal-winning Judo players of each team with a minimum age of 20 and a maximum age of 30. Consider players at the age of 20 and 30 too. Output the team along with the average player weight.

Table: olympics_athletes_events

# 56.Find players who participated in the Olympics representing more than one team


ESPN
Easy
ID 10143

Find players who participated in the Olympics representing more than one team.
Output the player name, team, games, sport, and the medal.

Table: olympics_athletes_events

# 57.Apple Product Counts


Google
Apple
Medium
ID 10141

Find the number of Apple product users and the number of total users with a device and group the counts by language. Assume Apple products are only MacBook-Pro, iPhone 5s, and iPad-air. Output the language along with the total number of Apple users and users with any device. Order your results based on the number of total users in descending order.

Tables: playbook_events, playbook_users

# 58. MacBook Pro Events


Google
Apple
Medium
ID 10140

Find how many events happened on MacBook-Pro per company in Argentina from users that do not speak Spanish.
Output the company id, language of users, and the number of events performed by users.

Tables: playbook_events, playbook_users

# 59.Number of Speakers By Language


Google
Apple
Medium
ID 10139

Find the number of speakers of each language by country. Output the country, language, and the corresponding number of speakers. Output the result based on the country in ascending order.

Tables: playbook_events, playbook_users

# 60.Even-numbered IDs Hired in June


Amazon
Bosch
Easy
ID 10137

Find employees who started in June and have even-numbered employee IDs.

Table: worker

# 61.Odd-numbered ID's Hired in February


Amazon
Bosch
Easy

Find employees who started in February and have odd-numbered employee IDs.

Table: worker

# 62.Spam Posts


Interview Question Date: June 2020

Meta/Facebook
Medium
ID 10134

Calculate the percentage of spam posts in all viewed posts by day. A post is considered a spam if a string "spam" is inside keywords of the post. Note that the facebook_posts table stores all posts posted by users. The facebook_post_views table is an action table denoting if a user has viewed a post.

Tables: facebook_posts, facebook_post_views

# 63.Requests Acceptance Rate


Airbnb
Medium
ID 10133

Find the acceptance rate of requests which is defined as the ratio of accepted contacts vs all contacts. Multiply the ratio by 100 to get the rate.

Table: airbnb_contacts

In [ ]:
整体/部分比值：
1. COUNT(ts.部分) / COUNT(整体)
2. SUM(CASE WHEN X THEN 1 ELSE 0 END) / COUNT(*)
3. COUNT(CASE WHEN X THEN A) / COUNT(*)

# 64. Highest Crime Rate


City of San Francisco
Easy
ID 10132

Find the number of crime occurrences for each day of the week.
Output the day alongside the corresponding crime count.

Table: sf_crime_incidents_2014_01

# 65. Business Name Lengths


Interview Question Date: June 2020

City of San Francisco
Hard
ID 10131


Find the number of words in each business name. Avoid counting special symbols as words (e.g. &). Output the business name and its count of words.

Table: sf_restaurant_health_violations

# 66.Find the number of inspections for each risk category by inspection type


Interview Question Date: June 2020

City of San Francisco
Medium
ID 10130
25

Data Engineer
Data Scientist
BI Analyst
Data Analyst
Find the number of inspections that resulted in each risk category per each inspection type.
Consider the records with no risk category value belongs to a separate category.
Output the result along with the corresponding inspection type and the corresponding total number of inspections per that type. The output should be pivoted, meaning that each risk category + total number should be a separate column.
Order the result based on the number of inspections per inspection type in descending order.

Table: sf_restaurant_health_violations

# 67. Count the number of movies that Abigail Breslin nominated for oscar


Google
Netflix
Easy
ID 10128

Count the number of movies that Abigail Breslin was nominated for an oscar.

Table: oscar_nominees

# 68. Calculate Samantha's and Lisa's total sales revenue


Amazon
Groupon
Salesforce
Easy
ID 10127

What is the total sales revenue of Samantha and Lisa?

Table: sales_performance

# 69.Bookings vs Non-Bookings


Interview Question Date: May 2020

Airbnb
Medium
ID 10124

Display the average number of times a user performed a search which led to a successful booking and the average number of times a user performed a search but did not lead to a booking. The output should have a column named action with values 'does not book' and 'books' as well as a 2nd column named average_searches with the average number of searches per action. Consider that the booking did not happen if the booking date is null. Be aware that search is connected to the booking only if their check-in dates match.

Tables: airbnb_contacts, airbnb_searches

# 70.Find the total number of searches for houses Westlake neighborhood with a TV


Airbnb
Easy
ID 10122

Find the total number of searches for houses in Westlake neighborhood with a TV among the amenities.

Table: airbnb_search_details

# 71.Number Of Custom Email Labels


Google
Medium
ID 10120

Find the number of occurrences of custom email labels for each user receiving an email. Output the receiver user id, label, and the corresponding number of occurrences.

Tables: google_gmail_emails, google_gmail_labels

# 72.User Exile


Meta/Facebook
Easy
ID 10091

Find the number of relationships that user  with id == 1 is not part of.

Table: facebook_friends

# 73.Find the percentage of shipable orders


Google
Amazon
Medium
ID 10090

Find the percentage of shipable orders.
Consider an order is shipable if the customer's address is known.

Tables: orders, customers

# 74. Find the number of customers without an order


Google
Amazon
Medium
ID 10089

Find the number of customers without an order.

Tables: orders, customers

# 75.Liked' Posts


Meta/Facebook
Medium
ID 10088

Find the number of posts which were reacted to with a like.

Tables: facebook_reactions, facebook_posts

# 76.Find all posts which were reacted to with a heart


Meta/Facebook
Easy
ID 10087

Find all posts which were reacted to with a heart. For such posts output all columns from facebook_posts table.

Tables: facebook_reactions, facebook_posts

# 77.Email Details Based On Sends


Google
Medium
ID 10086

Find all records from days when the number of distinct users receiving emails was greater than the number of distinct users sending emails

Table: google_gmail_emails

# 78.Meta/Facebook Matching Users Pairs


Meta/Facebook
Medium
ID 10085

Find matching pairs of Meta/Facebook employees such that they are both of the same nation, different age, same gender, and at different seniority levels.
Output ids of paired employees.

Table: facebook_employees

# 79. Cum Sum Energy Consumption


Interview Question Date: April 2020

Meta/Facebook
Hard
ID 10084

Calculate the running total (i.e., cumulative sum) energy consumption of the Meta/Facebook data centers in all 3 continents by the date. Output the date, running total energy consumption, and running total percentage rounded to the nearest whole number.

Tables: fb_eu_energy, fb_na_energy, fb_asia_energy

# 80.Start Dates Of Top Drivers


Lyft
Medium
ID 10083

Find contract starting dates of the top 5 most paid Lyft drivers. Consider only drivers who are still working with Lyft.

Table: lyft_drivers

# 81. Find the number of employees who received the bonus and who didn't


Microsoft
Dell
Hard
ID 10081

Find the number of employees who received the bonus and who didn't. Bonus values in employee table are corrupted so you should use  values from the bonus table. Be aware of the fact that employee can receive more than bonus.
Output value inside has_bonus column (1 if they had bonus, 0 if not) along with the corresponding number of employees for each.

Tables: employee, bonus

# 82. Find matching hosts and guests in a way that they are both of the same gender and nationality


Airbnb
Medium
ID 10078

Find matching hosts and guests pairs in a way that they are both of the same gender and nationality.
Output the host id and the guest id of matched pair.

Tables: airbnb_hosts, airbnb_guests

# 83.Income By Title and Gender


City of San Francisco
Medium
ID 10077

Find the average total compensation based on employee titles and gender. Total compensation is calculated by adding both the salary and bonus of each employee. However, not every employee receives a bonus so disregard employees without bonuses in your calculation. Employee can receive more than one bonus.
Output the employee title, gender (i.e., sex), along with the average total compensation.

Tables: sf_employee, sf_bonus

# 84.Find the average age of guests reviewed by each host


Airbnb
Medium
ID 10074

Find the average age of guests reviewed by each host.
Output the user along with the average age.

Tables: airbnb_reviews, airbnb_guests

# 85. Favorite Host Nationality


Interview Question Date: April 2020

Airbnb
Medium
ID 10073

For each guest reviewer, find the nationality of the reviewer’s favorite host based on the guest’s highest review score given to a host. Output the user ID of the guest along with their favorite host’s nationality. In case there is more than one favorite host from the same country, list that country only once (remove duplicates).


Both the from_user and to_user columns are user IDs.

Tables: airbnb_reviews, airbnb_hosts

# 86.Guest Or Host Kindness


Interview Question Date: April 2020

Airbnb
Easy
ID 10072

Find whether hosts or guests give higher review scores based on their average review scores. Output the higher of the average review score rounded to the 2nd decimal spot (e.g., 5.11).

Table: airbnb_reviews

# 87.Hosts' Abroad Apartments


Airbnb
Medium
ID 10071

Find the number of hosts that have accommodations in countries of which they are not citizens.

Tables: airbnb_hosts, airbnb_apartments

# 88.DeepMind employment competition


Google
Medium
ID 10070
14

Data Engineer
Data Scientist
BI Analyst
Data Analyst
Find the winning teams of DeepMind employment competition.
Output the team along with the average team score.
Sort records by the team score in descending order.

Tables: google_competition_participants, google_competition_scores

# 89.Correlation Between E-mails And Activity Time


Interview Question Date: April 2020

Google
Hard
ID 10069

There are two tables with user activities. The google_gmail_emails table contains information about emails being sent to users. Each row in that table represents a message with a unique identifier in the id field. The google_fit_location table contains user activity logs from the Google Fit app.
Find the correlation between the number of emails received and the total exercise per day. The total exercise per day is calculated by counting the number of user sessions per day.

Tables: google_gmail_emails, google_fit_location

# 90. User Email Labels


Interview Question Date: April 2020

Google
Medium
ID 10068

Find the number of emails received by each user under each built-in email label. The email labels are: 'Promotion', 'Social', and 'Shopping'. Output the user along with the number of promotion, social, and shopping mails count,.

Tables: google_gmail_emails, google_gmail_labels
Hints
Expected Output
All required columns and the first 5 rows of the solution are shown

# 91. Google Fit User Tracking


# 92. Fans vs Opposition


Interview Question Date: March 2020

Meta/Facebook
Hard
ID 10062

Meta/Facebook is quite keen on pushing their new programming language Hack to all their offices. They ran a survey to quantify the popularity of the language and send it to their employees. To promote Hack they have decided to pair developers which love Hack with the ones who hate it so the fans can convert the opposition. Their pair criteria is to match the biggest fan with biggest opposition, second biggest fan with second biggest opposition, and so on. Write a query which returns this pairing. Output employee ids of paired employees. Sort users with the same popularity value by id in ascending order.


Duplicates in pairings can be left in the solution. For example, (2, 3) and (3, 2) should both be in the solution.

Table: facebook_hack_survey

# 93. Find the number of reviews received by Lo-Lo's Chicken & Waffles for each star


Yelp
Easy
ID 10058

Find the number of reviews received by Lo-Lo's Chicken & Waffles for each star.
Output the number of stars along with the corresponding number of reviews.
Sort records by stars in ascending order.

Table: yelp_reviews

# 94.Popularity of Hack


Interview Question Date: March 2020

Meta/Facebook
Easy
ID 10061

Meta/Facebook has developed a new programing language called Hack.To measure the popularity of Hack they ran a survey with their employees. The survey included data on previous programing familiarity as well as the number of years of experience, age, gender and most importantly satisfaction with Hack. Due to an error location data was not collected, but your supervisor demands a report showing average popularity of Hack by office location. Luckily the user IDs of employees completing the surveys were stored.
Based on the above, find the average popularity of the Hack per office location.
Output the location along with the average popularity.

Tables: facebook_employees, facebook_hack_survey

# 95. Popularity of Hack


Interview Question Date: March 2020

Meta/Facebook
Easy
ID 10061

Meta/Facebook has developed a new programing language called Hack.To measure the popularity of Hack they ran a survey with their employees. The survey included data on previous programing familiarity as well as the number of years of experience, age, gender and most importantly satisfaction with Hack. Due to an error location data was not collected, but your supervisor demands a report showing average popularity of Hack by office location. Luckily the user IDs of employees completing the surveys were stored.
Based on the above, find the average popularity of the Hack per office location.
Output the location along with the average popularity.

Tables: facebook_employees, facebook_hack_survey

# Top Cool Votes


Interview Question Date: March 2020

Yelp
Medium

Find the review_text that received the highest number of  'cool' votes.
Output the business name along with the review text with the highest numbef of 'cool' votes.

Table: yelp_reviews

# Find the number of reviews received by Lo-Lo's Chicken & Waffles for each star


Yelp
Easy
ID 10058

Find the number of reviews received by Lo-Lo's Chicken & Waffles for each star.
Output the number of stars along with the corresponding number of reviews.
Sort records by stars in ascending order.

Table: yelp_reviews

# Most Checkins


Yelp
Medium
ID 10053

Find the top 5 businesses with the most check-ins.
Output the business id along with the number of check-ins.

Table: yelp_checkin

# Find the average number of stars for each state


Yelp
Easy
ID 10052

Find the average number of stars for each state.
Output the state name along with the corresponding average number of stars.

Table: yelp_business

# Find the number of open businesses


Yelp
Easy
ID 10051

Find the number of open businesses.

Table: yelp_business

# Find the review count for one-star businesses from yelp


Yelp
Easy
ID 10050

Find the review count for one-star businesses from yelp.
Output the name along with the corresponding review count.

Table: yelp_business

# Reviews of Categories


Interview Question Date: March 2020

Yelp
Medium
ID 10049

Find the top business categories based on the total number of reviews. Output the category along with the total number of reviews. Order by total reviews in descending order.

Table: yelp_business

# Top Businesses With Most Reviews


Yelp
Medium
ID 10048

Find the top 5 businesses with most reviews. Assume that each row has a unique business_id such that the total reviews for each business is listed on each row. Output the business name along with the total number of reviews and order your results by the total reviews in descending order.

Table: yelp_business

In [ ]:
--top 5 biz with most reviews
--unique biz_id ~ total reviews for each biz
--output: name, review_count： business name along with the total number of reviews 
--order by total reviews desc

--LIMIT 5 不好, rank 求top 5才最精准：
WITH business_rank AS(
SELECT 
    name,
    review_count,
    RANK() OVER (ORDER BY review_count DESC) AS rnk
FROM yelp_business
)
SELECT 
    name,
    review_count
FROM business_rank
WHERE rnk <= 5

# Top 5 States With 5 Star Businesses


Interview Question Date: March 2020

Yelp
Hard
ID 10046

Find the top 5 states with the most 5 star businesses. Output the state name along with the number of 5-star businesses and order records by the number of 5-star businesses in descending order. In case there are ties in the number of businesses, return all the unique states. If two states have the same result, sort them in alphabetical order.

Table: yelp_business

# Highest Priced Wine In The US


Interview Question Date: March 2020

Wine Magazine
Medium
ID 10044
13

Data Engineer
Data Scientist
BI Analyst
Data Analyst
Find the highest price in US country for each variety produced in English speaking regions, but not in Spanish speaking regions, with taking into consideration varieties that have earned a minimum of 90 points for every country they're produced in.
Output both the variety and the corresponding highest price.


Let's assume the US is the only English speaking region in the dataset, and Spain, Argentina are the only Spanish speaking regions in the dataset. Let's also assume that the same variety might be listed under several countries so you'll need to remove varieties that show up in both the US and in Spanish speaking countries.

Table: winemag_p1

# Median Price Of Wines


Interview Question Date: March 2020

Wine Magazine
Hard
ID 10043

Find the median price for each wine variety across both datasets. Output distinct varieties along with the corresponding median price.

Tables: winemag_p1, winemag_p2

# Top 3 Wineries In The World


Interview Question Date: March 2020

Wine Magazine
Hard
ID 10042

Find the top 3 wineries in each country based on the average points earned. In case there is a tie, order the wineries by winery name in ascending order. Output the country along with the best, second best, and third best wineries. If there is no second winery (NULL value) output 'No second winery' and if there is no third winery output 'No third winery'. For outputting wineries format them like this: "winery (avg_points)"

Table: winemag_p1

# Most Expensive And Cheapest Wine


Interview Question Date: March 2020

Wine Magazine
Hard
ID 10041

Find the cheapest and the most expensive variety in each region. Output the region along with the corresponding most expensive and the cheapest variety. Be aware that there are 2 region columns, the price from that row applies to both of them.


Note: The results set contains no ties.

Table: winemag_p1

# Find all wines from the winemag_p2 dataset which are produced in countries that have the highest sum of points in the winemag_p1 dataset


Wine Magazine
Hard
ID 10040

Find all wines from the winemag_p2 dataset which are produced in the country that have the highest sum of points in the winemag_p1 dataset.

Tables: winemag_p1, winemag_p2

# Macedonian Vintages


Wine Magazine
Medium
ID 10039

Find the vintage years of all wines from the country of Macedonia. The year can be found in the 'title' column. Output the wine (i.e., the 'title') along with the year. The year should be a numeric or int data type.

Table: winemag_p2

# Find all provinces which produced more wines in 'winemag_p1' than they did in 'winemag_p2'


Interview Question Date: March 2020

Wine Magazine
Medium
ID 10038

Find all provinces which produced more wines in 'winemag_p1' than they did in 'winemag_p2'.
Output the province and the corresponding wine count.
Order records by the wine count in descending order.

Tables: winemag_p1, winemag_p2

# Find Favourite Wine Variety


Interview Question Date: March 2020

Wine Magazine
Hard
ID 10037

Find each taster's favorite wine variety.
Consider that favorite variety means the variety that has been tasted by most of the time.
Output the taster's name along with the wine variety.

Table: winemag_p2

# Find the number of wines with and without designations per country


Interview Question Date: March 2020

Wine Magazine
Medium
ID 10035

Find the number of wines with and without designations per country.
Output the country along with the total without designations, total with designations, and the final total of both.

Table: winemag_p2

# Wine Variety Revenues


Interview Question Date: February 2020

Wine Magazine
Medium
ID 10033

Find the total revenue made by each region from each variety of wine in that region. Output the region, variety, and total revenue.


Take into calculation both region_1 and region_2. Remove the duplicated rows where  region, price and variety are exactly the same.

Table: winemag_p1

# Best Wines By Points-To-Price


Interview Question Date: February 2020

Wine Magazine
Medium
ID 10032

Find the wine with the highest points to price ratio. Output the title, points, price, and the corresponding points-to-price ratio.

Table: winemag_p2

# Find the number of Bodegas outside of Spain that produce wines with the blackberry taste


Interview Question Date: February 2020

Wine Magazine
Medium
ID 10031

Find the number of Bodegas (wineries with "bodega" pattern inside the name) outside of Spain that produce wines with the blackberry taste (description contains blackberry string). Group the count by country and region.
Output the country, region along with the number of bodegas.
Order records by the number of bodegas in descending order.

Table: winemag_p1

# Price Of Wines In Each Country


Interview Question Date: February 2020

Wine Magazine
Medium
ID 10029

Find the minimum, average, and maximum price of all wines per country. Assume all wines listed across both datasets are unique. Output the country name along with the corresponding minimum, maximum, and average prices.

Tables: winemag_p1, winemag_p2

# Find the number of wines each taster tasted within the variation


Interview Question Date: February 2020

Wine Magazine
Medium
ID 10028

Find the number of wines each taster tasted within the variation.
Output the tester's name, variety, and the number of tastings.
Order records by taster name and the variety in ascending order and by the number of tasting in descending order.

Table: winemag_p2

# Find the number of US-based wineries that have expensive wines (price >= 200)


Wine Magazine
Easy
ID 10027

Find the number of US-based wineries that have expensive wines.
A wine is considered to be expensive if its price is $200 or more.

Table: winemag_p1

# Find all wineries which produce wines by possessing aromas of plum, cherry, rose, or hazelnut


Wine Magazine
Medium
ID 10026

Find all wineries which produce wines by possessing aromas of plum, cherry, rose, or hazelnut. To make it more simple, look only for singular form of the mentioned aromas. HINT: if one of the specified words is just a substring of another word, this should not be a hit, but a miss.


Example Description: Hot, tannic and simple, with cherry jam and currant flavors accompanied by high, tart acidity and chile-pepper alcohol heat.
Therefore the winery Bella Piazza is expected in the results.

Table: winemag_p1

# Find all possible varieties which occur in either of the winemag datasets


Wine Magazine
Medium
ID 10025

Find all possible varieties which occur in either of the winemag datasets.
Output unique variety values only.
Sort records based on the variety in ascending order.

Tables: winemag_p1, winemag_p2

# Wine varieties tasted by 'Roger Voss'


Wine Magazine
Easy
ID 10024

Find wine varieties tasted by 'Roger Voss' and with a value in the 'region_1' column of the dataset. Output unique variety names only.

Table: winemag_p2

# Find all wine varieties which can be considered cheap based on the price


Wine Magazine
Easy
ID 10022

Find all wine varieties which can be considered cheap based on the price.
A variety is considered cheap if the price of a bottle lies between 5 to 20 USD.
Output unique variety names only.

Table: winemag_p1

# Find all top-rated wineries based on points


Wine Magazine
Easy
ID 10021

Find all top-rated wineries based on points.
Consider a top-rated winery has been awarded points more or equal than 95.

Table: winemag_p1

# Find prices for Spanish, Italian, and French wines


Wine Magazine
Easy
ID 10020

Find prices for Spanish, Italian, and French wines. Output the price.

Table: winemag_p1

# Find the fraction of rides for each weather and the hour


Interview Question Date: February 2020

Lyft
Hard
ID 10019

Find the fraction (percentage divided by 100) of rides each weather-hour combination constitutes among all weather-hour combinations.
Output the weather, hour along with the corresponding fraction.

Table: lyft_rides